# Item Based Recommendation 

Recommendation based on track similarity. A track is similar to another one if it is listened by the same user.

In [2]:
import pandas as pd
import numpy as np
import sys
from scipy.sparse import *
from scipy import *
import pickle

In [3]:
# read test set
training = pd.read_csv("training.csv")[['user', 'track']]
training.head()

,user,track
0,000c556aa021426138c47076d53dfbec,spotify:track:000UQuSa2QcsWFK0tGTBDM
1,000c556aa021426138c47076d53dfbec,spotify:track:001faoWrA2TK6dw9vHp9p6
2,000c556aa021426138c47076d53dfbec,spotify:track:010jenmHogRDArWKod52XR
3,000c556aa021426138c47076d53dfbec,spotify:track:01gijlGB36OqjXJ0bUkE99
4,000c556aa021426138c47076d53dfbec,spotify:track:01z7U4QxORx59DIfSwImo2


In [4]:
# read test set
tracks = training['track'].unique()
size = len(tracks)
#items = np.zeros((size,size))
items_sparse = csr_matrix( (size,size), dtype=int16 )
track_id_dict = {}
id_track_dict = {}
for i,track in enumerate(tracks):
    track_id_dict[track] = i
    id_track_dict[i] = track
    
with open('track_to_id.pickle', 'wb') as handle:
    pickle.dump(track_id_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)    
with open('id_to_track.pickle', 'wb') as handle:
    pickle.dump(id_track_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)        

In [5]:
user_dict = {}
groups = training.groupby('user')
for i, group in groups:
    user_id = group['user'].iloc[0]
    track_ids = set(group['track'])
    user_dict[user_id] = track_ids    

In [8]:
count = 0
for tracks in user_dict.values():
    track_ids = [track_id_dict[track] for track in tracks]
    sys.stdout.write("User: "+str(count)+" --- "+str(len(track_ids))+" tracks \r")
    sys.stdout.flush()
    count += 1
    if len(track_ids) < 300 and len(track_ids) > 50:
        for track_id in track_ids:
            #print(track_id_dict[track_id])
            row = array([track_id for i in track_ids])
            col = array(track_ids)
            data = array([1 for i in track_ids])
            acc = csr_matrix( (data,(row,col)), shape=(size,size), dtype=int16 )
            items_sparse += acc

In [27]:
#items_sparse.size
#save_npz('items_sparse.npz', items_sparse)
#items_sparse.todense()
#from tempfile import TemporaryFile
#outfile = TemporaryFile()
#np.save(outfile, items)
len(find(items_sparse)[2])

#from scipy import sparse
#sparse = sparse.csr_matrix(items)

54050

## Suche

In [28]:
track_list = ['spotify:track:3eBLhHpqmsTlSoSw5gRxhB', 'spotify:track:01gijlGB36OqjXJ0bUkE99']
id_list = [track_id_dict[track] for track in track_list]

In [29]:
result_sparse = csr_matrix( (1,size), dtype=int8 )
for track_id in id_list:
    row_sparse = items_sparse.getrow(track_id)
    print(row_sparse.size)
    result_sparse += row_sparse

print("----------")
print(result_sparse.size)
#result = [(index, item) for index, item in enumerate(result)]
#result.sort(key=lambda item: item[1], reverse=True)
#result = result[:10]
#result = [(track_id_dict[i[0]]) for i in result]
#result

10
0
----------
10


In [35]:
indices = result_sparse.nonzero()[1].tolist()
values =  result_sparse.data.tolist()
zipped = list(zip(indices, values))
filtered = list(filter(lambda item: item[0] not in id_list, zipped))
filtered.sort(key=lambda item: item[1], reverse=True)
shortened = filtered[:5]
result = [id_track_dict[i[0]] for i in shortened]
result

['spotify:track:0qpTmi5o3LOX3PqMEd6s2F',
 'spotify:track:0v945WRZhamHsBBqbpW4LH',
 'spotify:track:46BtkjwDx6o6qr5fiSaO2D',
 'spotify:track:4APxFzhKBfGczlAn4jQJiM',
 'spotify:track:4OY3D0dh8vYO27gnquE6V1']

In [68]:
#my_dict = {'a':['x', 'y'], 'b':['z', 'y']}

In [69]:
#df = pd.DataFrame(columns=['track', 'user'])

In [66]:
#for key, tracks in my_dict.items():
#    size = len(tracks)
#    new = pd.DataFrame({'user': [key]*size,'track': tracks})
#    df = df.append(new)